In [9]:
import pandas as pd
import json
import spotipy
import spotipy.oauth2 as SpotifyClientCredentials

CLIENT_ID = '69200cfc38774a4394c7bc17911d0c08'
CLIENT_SECRET = '6c5f36f3da3e4e699ebc7ba59d3d9792'
REDIRECT_URI = "http://127.0.0.1:8080"

# auth_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
auth_manager= spotipy.oauth2.SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope='playlist-modify-public, playlist-modify-private')
sp = spotipy.Spotify(auth_manager=auth_manager) 

In [35]:
with open('./data/jsech_liked_songs.json') as f:
    jsechs_library = json.load(f)
    
with open('./data/nate_liked_songs.json') as f:
    nates_library = json.load(f)
    
with open('./data/jgreene_liked_songs.json') as f:
    jgreenes_library = json.load(f)

In [36]:
lib_list = []
for item in jgreenes_library:
    data = {}
    track = item['track']
    data['track_name'] = track['name']
    data['artists'] = ", ".join([artist['name'] for artist in track['artists']])
    data['album'] = track['album']['name']
    data['duration_ms'] = track['duration_ms']
    data['explicit'] = track['explicit']
    data['popularity'] = track['popularity']
    data['href'] = track['href']
    data['id'] = track['id']
    data['uri'] = track['uri']
    lib_list.append(data)

jgreene_df = pd.DataFrame(lib_list)

In [5]:
# Get Audio Features for the tracks in a library
def merge_audio_feats(library):
    audio_feats = []
    i=0
    while i < library.shape[0]:
        audio_feats += sp.audio_features(tracks=list(library['id'])[i:i+100])
        i += 100

    for i in range(len(audio_feats)):
        if audio_feats[i] == None:
            print(library.iloc[i]['track_name'])
            audio_feats[i] = {}

    audio_df = pd.DataFrame(audio_feats).drop(columns=['track_href', 'type', 'analysis_url'])
    library = pd.merge(library, audio_df, how="left", on=["id",'uri', 'duration_ms'])

    cols = [ 'track_name',
             'artists',
             'album',
             'popularity',
             'duration_ms',
             'explicit',
             'acousticness',
             'danceability',
             'energy',
             'instrumentalness',
             'key',
             'liveness',
             'loudness',
             'mode',
             'speechiness',
             'tempo',
             'time_signature',
             'valence',
             'href',
             'id',
             'uri']
    library = library[cols]
    return library

In [37]:
# jsech_df = merge_audio_feats(jsech_df)
# nate_df = merge_audio_feats(nate_df)
jgreene_df = merge_audio_feats(jgreene_df)
common_df = pd.merge(jsech_df,jgreene_df,how="inner")

In [38]:
common_df.shape[0]

81

In [20]:
category = "time_signature"
print(f"Justin: {jsech_df[category].mean()}")
print(f"Nate: {nate_df[category].mean()}")
print(f"Both: {common_df[category].mean()}")

Justin: 3.8446601941747574
Nate: 3.9053318824809575
Both: 3.875


In [39]:
playlist = sp.user_playlist_create(user='j.sech',name='Common Tracks - Justins')
sp.user_playlist_add_tracks(user='j.sech',playlist_id=playlist['uri'], tracks=list(common_df['id'])[:100])
# sp.user_playlist_add_tracks(user='j.sech',playlist_id=playlist['uri'], tracks=list(common_df['id'])[100:])

{'snapshot_id': 'Myw2MjU0ZTg2ZmYxMDc0ZDJiOWYwZWM5OWFmMTA5NTEyYTM1MzUxYTIx'}